# Amplifierd API - Enhanced Session Lifecycle

This notebook demonstrates the enhanced session management system with integrated mount plan generation.

## Overview

The enhanced session system provides:
- **Integrated mount plan generation**: Sessions automatically generate mount plans on creation
- **Complete lifecycle management**: CREATED → ACTIVE → COMPLETED/FAILED/TERMINATED
- **Transcript management**: Append-only JSONL for efficient message storage
- **Session queries**: Filter by status, profile, date range
- **State persistence**: Atomic file operations with session.json, transcript.jsonl, mount_plan.json
- **Cleanup utilities**: Age-based cleanup with status protection
- **Amplified directories**: Sessions can be created in specific working directories (see notebook 08)

**Session States**:
- `CREATED`: Session exists, mount plan generated, not yet started
- `ACTIVE`: Session running, messages being exchanged
- `COMPLETED`: Session finished successfully
- `FAILED`: Session ended with error
- `TERMINATED`: Session killed by user

**Storage Structure**:
```
.amplifierd/state/sessions/
  {session_id}/
    mount_plan.json    # Complete mount plan
    session.json       # Session metadata and state
    transcript.jsonl   # Message history (append-only)
```

**Note**: This notebook demonstrates sessions in the default (root) directory. For working with multiple project directories, see `08-amplified-directories.ipynb`.

In [1]:
# Setup and helper functions
import json
from typing import Any

import requests

BASE_URL = "http://127.0.0.1:8421"
API_BASE = f"{BASE_URL}/api/v1"


def print_response(response: requests.Response, label: str = "RESPONSE") -> Any:
    """Pretty-print API response with formatting."""
    print(f"\n{'=' * 60}")
    print(f"{label}")
    print(f"{'=' * 60}")
    print(f"Status: {response.status_code}")

    if response.status_code == 204:
        print("No content returned (successful operation)")
        return None

    try:
        data = response.json()
        print(json.dumps(data, indent=2))
        return data
    except Exception as e:
        print(f"Error parsing response: {e}")
        print(f"Raw response: {response.text}")
        return None


print("✓ Setup complete")
print(f"  API Base: {API_BASE}")

✓ Setup complete
  API Base: http://127.0.0.1:8421/api/v1


## Session Creation

Create a new session, which automatically generates a mount plan and initializes session state.

The session creation process:
1. Creates session directory structure
2. Generates mount plan from profile
3. Initializes session.json with CREATED state
4. Creates empty transcript.jsonl
5. Returns session metadata

**New in this version**: Sessions can now be created in specific amplified directories (optional `amplified_dir` parameter). This notebook uses the default root directory. See `08-amplified-directories.ipynb` for multi-directory examples.

In [ ]:
# Create session (automatically generates mount plan)
create_request = {
    "profile_name": "developer-expertise/dev",
    "settings_overrides": {"llm": {"model": "claude-3-5-sonnet-20241022"}},
    # "amplified_dir": "projects/myapp",  # Optional: specify working directory (defaults to root)
}

response = requests.post(f"{API_BASE}/sessions/", json=create_request)
session = print_response(response, "CREATE SESSION")

if session:
    session_id = session["sessionId"]
    print(f"\n✓ Session created: {session_id}")
    print(f"  Status: {session['status']}")
    print(f"  Profile: {session['profileName']}")
    print(f"  Amplified Dir: {session.get('amplifiedDir', '.')} (root)")
    print(f"  Message Count: {session['messageCount']}")
    print(f"  Created at: {session['createdAt']}")

## Session Lifecycle - Start Session

Transition the session from CREATED to ACTIVE state. This marks the beginning of actual work.

In [ ]:
# Start the session (CREATED → ACTIVE)
response = requests.post(f"{API_BASE}/sessions/{session_id}/start")
print_response(response, "START SESSION")

# Verify transition
response = requests.get(f"{API_BASE}/sessions/{session_id}")
updated_session = print_response(response, "VERIFY ACTIVE STATE")

if updated_session:
    print(f"\n✓ Status: {updated_session['status']}")
    print(f"  Started at: {updated_session.get('startedAt', 'N/A')}")
    print(f"  Duration: {updated_session.get('duration', 'N/A')}")

## Message Management

Demonstrate transcript operations:
- Appending messages to the transcript
- Retrieving the complete message history
- Automatic message counting

Messages are stored in an append-only JSONL file for efficiency and simplicity.

In [ ]:
# Append messages to transcript
messages = [
    {"role": "user", "content": "What is the capital of France?"},
    {"role": "assistant", "content": "The capital of France is Paris.", "tokenCount": 15},
    {"role": "user", "content": "What's the population?"},
    {
        "role": "assistant",
        "content": "Paris has a population of approximately 2.1 million people within the city limits, and over 12 million in the metropolitan area.",
        "tokenCount": 32,
    },
]

print("Adding messages to transcript...")
for msg in messages:
    response = requests.post(f"{API_BASE}/sessions/{session_id}/messages", json=msg)
    if response.status_code == 201:
        print(f"✓ Message added: [{msg['role']}] {msg['content'][:50]}...")
    else:
        print(f"✗ Failed to add message: {response.status_code}")

In [ ]:
# Get transcript
response = requests.get(f"{API_BASE}/sessions/{session_id}/transcript")
transcript = print_response(response, "GET TRANSCRIPT")

if transcript:
    print(f"\n✓ Retrieved {len(transcript)} messages")
    for i, msg in enumerate(transcript, 1):
        content_preview = msg["content"][:60] + "..." if len(msg["content"]) > 60 else msg["content"]
        print(f"  {i}. [{msg['role']:10s}] {content_preview}")
        if "tokenCount" in msg:
            print(f"      Tokens: {msg['tokenCount']}")

In [ ]:
# Verify message count updated in session
response = requests.get(f"{API_BASE}/sessions/{session_id}")
updated_session = print_response(response, "VERIFY MESSAGE COUNT")

if updated_session:
    print(f"\n✓ Session message count: {updated_session['messageCount']}")

## Session Queries

Demonstrate filtering and querying capabilities:
- List all sessions
- Filter by status (active, completed, etc.)
- Filter by profile name

In [ ]:
# List all sessions
response = requests.get(f"{API_BASE}/sessions/")
all_sessions = print_response(response, "LIST ALL SESSIONS")

if all_sessions:
    print(f"\n✓ Total sessions: {len(all_sessions)}")
    for sess in all_sessions:
        print(f"  • {sess['sessionId'][:8]}... - {sess['status']} - {sess['profileName']}")

In [ ]:
# List active sessions only
response = requests.get(f"{API_BASE}/sessions/?status=active")
active_sessions = print_response(response, "LIST ACTIVE SESSIONS")

if active_sessions:
    print(f"\n✓ Active sessions: {len(active_sessions)}")
    for sess in active_sessions:
        print(f"  • {sess['sessionId'][:8]}... - {sess['profileName']}")
        print(f"    Started: {sess.get('startedAt', 'N/A')}")
        print(f"    Messages: {sess['messageCount']}")

In [ ]:
# List sessions by profile
response = requests.get(f"{API_BASE}/sessions/?profileName=developer-expertise/dev")
profile_sessions = print_response(response, "FILTER BY PROFILE")

if profile_sessions:
    print(f"\n✓ Sessions for 'developer-expertise/dev': {len(profile_sessions)}")
    for sess in profile_sessions:
        print(f"  • {sess['sessionId'][:8]}... - Status: {sess['status']}")

## Complete Session

Mark the session as successfully completed. This transitions from ACTIVE to COMPLETED state.

In [ ]:
# Complete the session (ACTIVE → COMPLETED)
response = requests.post(f"{API_BASE}/sessions/{session_id}/complete")
print_response(response, "COMPLETE SESSION")

# Verify completion
response = requests.get(f"{API_BASE}/sessions/{session_id}")
final_session = print_response(response, "FINAL SESSION STATE")

if final_session:
    print("\n✓ Session completed")
    print(f"  Status: {final_session['status']}")
    print(f"  Message Count: {final_session['messageCount']}")
    print(f"  Started at: {final_session.get('startedAt', 'N/A')}")
    print(f"  Ended at: {final_session.get('endedAt', 'N/A')}")
    print(f"  Duration: {final_session.get('duration', 'N/A')}")

## Retrieve Mount Plan

Get the mount plan that was automatically generated when the session was created.

In [ ]:
# Get the mount plan for this session
response = requests.get(f"{API_BASE}/sessions/{session_id}/mount-plan")
session_mount_plan = print_response(response, "GET SESSION MOUNT PLAN")

if session_mount_plan:
    print("\n✓ Mount plan retrieved")
    print(f"  Profile: {session_mount_plan['profileName']}")
    print(f"  Agents: {len(session_mount_plan['agents'])}")
    print(f"  Context: {len(session_mount_plan['context'])}")
    print(f"  Total mount points: {len(session_mount_plan['mountPoints'])}")

    # Show some agents
    if session_mount_plan["agents"]:
        print("\n  Sample agents:")
        for agent in session_mount_plan["agents"][:3]:
            print(f"    • {agent['ref']}")

    # Show some context
    if session_mount_plan["context"]:
        print("\n  Sample context:")
        for ctx in session_mount_plan["context"][:3]:
            print(f"    • {ctx['ref']}")

## Error Scenarios

Demonstrate error handling:
- Creating a session that fails
- Recording error details
- Terminating a session

In [ ]:
# Test fail transition with error details
print("Creating a session to demonstrate failure handling...")
error_session_request = {"profile_name": "developer-expertise/dev"}
response = requests.post(f"{API_BASE}/sessions/", json=error_session_request)
error_session = response.json()
error_session_id = error_session["sessionId"]
print(f"✓ Created session: {error_session_id[:8]}...")

# Start it
response = requests.post(f"{API_BASE}/sessions/{error_session_id}/start")
print("✓ Started session")

# Mark as failed
fail_request = {
    "error_message": "Test error: Something went wrong during processing",
    "error_details": {"code": "TEST_ERROR", "context": "Demo notebook error scenario", "line": 42},
}
response = requests.post(f"{API_BASE}/sessions/{error_session_id}/fail", json=fail_request)
print_response(response, "FAIL SESSION")

In [ ]:
# Verify error recorded
response = requests.get(f"{API_BASE}/sessions/{error_session_id}")
failed_session = print_response(response, "FAILED SESSION STATE")

if failed_session:
    print("\n✓ Error recorded")
    print(f"  Status: {failed_session['status']}")
    print(f"  Error: {failed_session.get('errorMessage', 'N/A')}")
    print(f"  Details: {failed_session.get('errorDetails', 'N/A')}")
    print(f"  Ended at: {failed_session.get('endedAt', 'N/A')}")

In [ ]:
# Test terminate transition
print("\nCreating another session to demonstrate termination...")
terminate_session_request = {"profile_name": "developer-expertise/dev"}
response = requests.post(f"{API_BASE}/sessions/", json=terminate_session_request)
terminate_session = response.json()
terminate_session_id = terminate_session["sessionId"]
print(f"✓ Created session: {terminate_session_id[:8]}...")

# Start it
response = requests.post(f"{API_BASE}/sessions/{terminate_session_id}/start")
print("✓ Started session")

# Terminate it
response = requests.post(f"{API_BASE}/sessions/{terminate_session_id}/terminate")
print_response(response, "TERMINATE SESSION")

# Verify termination
response = requests.get(f"{API_BASE}/sessions/{terminate_session_id}")
terminated_session = print_response(response, "TERMINATED SESSION STATE")

if terminated_session:
    print("\n✓ Session terminated")
    print(f"  Status: {terminated_session['status']}")
    print(f"  Ended at: {terminated_session.get('endedAt', 'N/A')}")

## Cleanup

Delete the test sessions created in this notebook.

In [ ]:
# Delete test sessions
print("Cleaning up test sessions...")
for sid in [session_id, error_session_id, terminate_session_id]:
    response = requests.delete(f"{API_BASE}/sessions/{sid}")
    if response.status_code == 204:
        print(f"✓ Deleted session: {sid[:8]}...")
    else:
        print(f"✗ Failed to delete session {sid[:8]}...: {response.status_code}")

## Summary

This notebook demonstrated the complete enhanced session lifecycle:

### Key Features

1. **Session Creation**
   - Automatic mount plan generation
   - Profile-based initialization
   - Settings override support
   - **NEW**: Optional amplified directory specification

2. **Lifecycle Management**
   - CREATED → ACTIVE → COMPLETED/FAILED/TERMINATED
   - State validation and transitions
   - Timestamp tracking

3. **Message Management**
   - Append-only transcript storage
   - Automatic message counting
   - Token tracking support

4. **Query Capabilities**
   - Filter by status
   - Filter by profile
   - List all sessions

5. **Error Handling**
   - Structured error recording
   - Error details persistence
   - Graceful failure modes

### API Reference

| Endpoint | Method | Description |
|----------|--------|-------------|
| `/sessions/` | POST | Create new session with mount plan |
| `/sessions/` | GET | List sessions (with filters) |
| `/sessions/{id}` | GET | Get session details |
| `/sessions/{id}` | DELETE | Delete session |
| `/sessions/{id}/start` | POST | Start session (CREATED → ACTIVE) |
| `/sessions/{id}/complete` | POST | Complete session (ACTIVE → COMPLETED) |
| `/sessions/{id}/fail` | POST | Mark session as failed (ACTIVE → FAILED) |
| `/sessions/{id}/terminate` | POST | Terminate session (ACTIVE → TERMINATED) |
| `/sessions/{id}/messages` | POST | Append message to transcript |
| `/sessions/{id}/transcript` | GET | Get all messages |
| `/sessions/{id}/mount-plan` | GET | Get session's mount plan |

**NEW Parameters**:
- `amplified_dir` (optional) - Specify working directory context (defaults to root)
- See `08-amplified-directories.ipynb` for details

### Session States

```
CREATED
  ↓ (start)
ACTIVE
  ↓ (complete/fail/terminate)
COMPLETED | FAILED | TERMINATED
```

### Storage Structure

```
.amplifierd/state/sessions/{session_id}/
  ├── mount_plan.json    # Generated from profile
  ├── session.json       # Session metadata and state
  └── transcript.jsonl   # Message history (one per line)
```

### Next Steps

- See `02-sessions-and-messages.ipynb` for original session patterns
- See `03-profile-management.ipynb` for profile system
- **NEW**: See `08-amplified-directories.ipynb` for multi-directory context management
- See API documentation for full endpoint details